<a href="https://colab.research.google.com/github/heriswaya/MLT-Project-1-and-2/blob/main/rekomendasi_barang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Understanding

In [8]:
# Unduh file zip dari GitHub
!wget https://github.com/heriswaya/MLT-Project-1-and-2/raw/main/dataset_recommendation/archive.zip

--2025-06-01 03:40:12--  https://github.com/heriswaya/MLT-Project-1-and-2/raw/main/dataset_recommendation/archive.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/heriswaya/MLT-Project-1-and-2/main/dataset_recommendation/archive.zip [following]
--2025-06-01 03:40:13--  https://raw.githubusercontent.com/heriswaya/MLT-Project-1-and-2/main/dataset_recommendation/archive.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6957675 (6.6M) [application/zip]
Saving to: ‘archive.zip’

archive.zip         100%[===================>]   6.63M  --.-KB/s    in 0.09s   

2025-06-01 03:40:13 (73.6 MB/s) - ‘ar

In [9]:
# Ekstrak file zip
!unzip archive.zip

Archive:  archive.zip
  inflating: campaigns.csv           
  inflating: customer_reviews_complete.csv  
  inflating: customers.csv           
  inflating: interactions.csv        
  inflating: support_tickets.csv     
  inflating: transactions.csv        


Terdapat 6 file csv. <br>
Untuk membangun model Collaborative Filtering, hanya perlu menggunakan file-file yang mengandung interaksi eksplisit atau implisit antara user dan item. Berikut file yang akan digunakan:
Untuk membangun model **Collaborative Filtering**, Anda hanya perlu menggunakan file-file yang mengandung **interaksi eksplisit atau implisit antara user dan item**. Berikut rekomendasi file yang akan digunakan:
1. **`transactions.csv`**  
   - **Alasan**: Ini adalah data transaksi aktual yang menunjukkan pembelian produk oleh pelanggan.
   - **Kolom Penting**:
     - `customer_id` (User)
     - `product_name` (Item)
     - `quantity` (Implicit rating: bisa dianggap sebagai "strength of interaction")
     - `price` (Opsional: bisa digunakan untuk weighting)

2. **`customer_reviews_complete.csv`**  
   - **Alasan**: Berisi rating eksplisit (1-5) dari pelanggan.
   - **Kolom Penting**:
     - `customer_id` (User)
     - `product_name` (Item)
     - `rating` (Explicit rating)

3. **`interactions.csv`** (Opsional)  
   - **Alasan**: Bisa digunakan sebagai implicit feedback tambahan.
   - **Kolom Penting**:
     - `customer_id` (User)
     - `page_or_product` (Item)
     - `interaction_type` (e.g., "View Product" = 1 point, "Add to Cart" = 2 points, dll.)

### File yang Tidak Perlu Digunakan:
- `customers.csv` (Data demografi - lebih cocok untuk content-based, bukan collaborative)
- `campaigns.csv` (Info marketing - tidak relevan)
- `support_tickets.csv` (Data support - tidak relevan)

In [20]:
# Import library
import pandas as pd

# Membaca data-data yang relevan untuk collaborative filtering
transactions = pd.read_csv('/content/transactions.csv')
reviews = pd.read_csv('/content/customer_reviews_complete.csv')
interactions = pd.read_csv('/content/interactions.csv')
customers = pd.read_csv('/content/customers.csv')  # Untuk referensi saja

In [21]:
transactions.head()

,transaction_id,customer_id,product_name,product_category,quantity,price,transaction_date,store_location,payment_method,discount_applied
0,53e3d9f5-6231-46de-aa39-e5f8eaae2eee,727839b2-f084-4e94-94d8-ae59cc8e4b84,Ring Doorbell,Smart Home Devices,1.0,140.07,2020-12-06,"Houston, TX",Credit Card,20.0
1,ae6e6edf-f2fe-45c8-989c-c01e77eade0c,c4ac244c-989e-46fb-b243-91dd1d5cf739,Oculus Quest,Gaming Consoles,1.0,339.57,2021-04-26,Online,Gift Card,0.0
2,ba85240a-2b5d-4987-ac0a-efb6a4a57d6d,d296bf0d-92bb-4670-a578-2fd3553cf7c8,Samsung Galaxy S22,Smartphones,2.0,1335.88,2021-10-28,Online,PayPal,30.0
3,3725a285-2a2b-4a80-b317-7c8041cd9b8b,7b1b9980-9d65-40fc-8ad6-6afaae4699e4,External Hard Drive,Computer Accessories,1.0,91.56,2023-03-16,Online,Credit Card,25.0
4,de1b05a5-68c7-4b00-a068-e8c2f13f0dab,8595dcfa-2bfc-46f4-a15a-042af85b78cd,MacBook Pro,Laptops,1.0,985.15,2022-08-30,"Atlanta, GA",Credit Card,0.0


In [22]:
reviews.head()

,review_id,customer_id,product_name,product_category,full_name,transaction_date,review_date,rating,review_title,review_text
0,rev_000001,e9848049-323f-4f95-a26e-3cffc219d591,Bookshelf,Furniture,Patrick Guerrero,2024-05-30,2024-06-12,5,Sturdy and Elegant Addition to My Study Room,Just received the bookshelf I've been eyeing f...
1,rev_000002,24bcc3c6-ce17-423f-87f5-a1f6d1114ba7,Oculus Quest,Gaming Consoles,Katie Burton,2024-03-10,2024-04-05,5,Immersive Virtual Reality Adventure!,"Katie Burton, a tech enthusiast from Dallas, T..."
2,rev_000003,8806399f-1c9b-47f7-b8ca-fac5468438ff,OnePlus 10,Smartphones,Fernando Rogers,2024-06-14,2024-06-29,4,"Impressive Performance, Room for Improvement i...",The OnePlus 10 has been a great addition to my...
3,rev_000004,62ab496d-f69a-48af-b44c-08ac79b6f68d,iPhone 13,Smartphones,Sara Howell,2024-03-27,2024-04-06,4,"Solid Upgrade, Could've Been Perfect!",The iPhone 13 has been a great addition to my ...
4,rev_000005,1b7daed4-aaa6-41b1-9826-0d51d67614e3,Bookshelf,Furniture,Kelly Kim,2023-08-16,2023-09-01,4,"Sturdy, Yet Lacking Personality",I recently purchased this bookshelf for my hom...


In [23]:
interactions.head()

,interaction_id,customer_id,channel,interaction_type,interaction_date,duration,page_or_product,session_id
0,423db3d3-659d-433f-859a-eea2ff1e2d2c,00012aa8-e99c-4e30-b3f6-1f7e36adc517,NaN,review,2023-03-11 08:37:00,128.0,home,00012aa8-e99c-4e30-b3f6-1f7e36adc517_session_0
1,8cf67dd6-63f8-4333-924d-2126aa87a91d,00012aa8-e99c-4e30-b3f6-1f7e36adc517,web,wishlist_add,2023-03-22 04:57:00,9.0,Dell XPS 15,00012aa8-e99c-4e30-b3f6-1f7e36adc517_session_1
2,2e8e17c8-c40f-46ea-810f-08c987eb07c7,00012aa8-e99c-4e30-b3f6-1f7e36adc517,web,review,2023-04-11 14:47:00,205.0,category_kitchen,00012aa8-e99c-4e30-b3f6-1f7e36adc517_session_2
3,d5d18036-735c-4d70-842a-0f82a4873fbf,00012aa8-e99c-4e30-b3f6-1f7e36adc517,web,page_view,2023-05-23 07:20:00,65.0,category_laptops,00012aa8-e99c-4e30-b3f6-1f7e36adc517_session_3
4,ef756343-1635-4510-85c8-41a399adb8f8,00012aa8-e99c-4e30-b3f6-1f7e36adc517,web,product_view,2023-06-06 02:57:00,146.0,Bose Headphones,00012aa8-e99c-4e30-b3f6-1f7e36adc517_session_4


In [24]:
customers.head()

,customer_id,full_name,age,gender,email,phone,street_address,city,state,zip_code,registration_date,preferred_channel
0,4c30e132-0704-4459-a509-9eddde934977,Mark Johnson,40.0,Male,mark.johnson@yahoo.com,989.608.3863,819 Johnson Course,Houston,Texas,29158.0,2024-04-25,NaN
1,68bec407-275f-4b5b-9a82-13d02f54626a,Robert Smith,33.0,Male,smithr@yahoo.com,(518)349-5931x0341,35116 Michael Key Suite 078,Austin,Texas,16862.0,2021-05-30,in-store
2,4466459f-76c8-433c-814e-6d59cb4131fc,Jamie Chavez,42.0,Female,jchavez@gmail.com,364.583.5030x564,419 Amanda Gardens,Detroit,Michigan,21918.0,2023-12-14,online
3,04c36a25-02f3-462c-92b0-6bf291c57706,Thomas Bradley,53.0,Male,thomas.bradley@hotmail.com,(332)887-1012x269,7242 Julie Plain Suite 969,Fort Worth,Texas,52851.0,2022-07-11,both
4,e916df3d-c3f5-40b0-8ae2-5d043be88300,Jane Ferrell,32.0,Female,jane.ferrell@hotmail.com,5484281489,845 Kelly Estate,Atlanta,Georgia,59971.0,2020-09-06,online


In [25]:
print('Jumlah data pelanggan unik: ', len(customers.customer_id.unique()))
print('Jumlah produk unik yang dibeli: ', len(transactions.product_name.unique()))
print('Jumlah transaksi: ', len(transactions))
print('Jumlah rating/review: ', len(reviews))
print('Jumlah interaksi pengguna: ', len(interactions))

print('\n=== DETAIL INTERAKSI ===')
print('Jumlah pelanggan yang memberikan rating: ', len(reviews.customer_id.unique()))
print('Jumlah produk yang mendapat rating: ', len(reviews.product_name.unique()))
print('Jumlah pelanggan yang berinteraksi: ', len(interactions.customer_id.unique()))
print('Jumlah produk yang diinteraksi: ', len(interactions.page_or_product.unique()))

print('\n=== DISTRIBUSI RATING ===')
print(reviews.rating.value_counts().sort_index())

Jumlah data pelanggan unik:  5000
Jumlah produk unik yang dibeli:  76
Jumlah transaksi:  32295
Jumlah rating/review:  1000
Jumlah interaksi pengguna:  100000

=== DETAIL INTERAKSI ===
Jumlah pelanggan yang memberikan rating:  830
Jumlah produk yang mendapat rating:  75
Jumlah pelanggan yang berinteraksi:  4994
Jumlah produk yang diinteraksi:  35

=== DISTRIBUSI RATING ===
rating
1     30
2     76
3    142
4    350
5    402
Name: count, dtype: int64


## Univariate Exploratory Data Analysis

Penjelasan nama variabel yang digunakan:
- transactions: Data transaksi pembelian produk oleh pelanggan
- reviews: Data rating dan review produk oleh pelanggan
- interactions: Data interaksi pengguna dengan produk (view, click, dll)
- customers: Data profil pelanggan (digunakan hanya untuk referensi)

### Transactions

In [26]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32295 entries, 0 to 32294
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    32295 non-null  object 
 1   customer_id       32295 non-null  object 
 2   product_name      31617 non-null  object 
 3   product_category  31609 non-null  object 
 4   quantity          31651 non-null  float64
 5   price             31673 non-null  float64
 6   transaction_date  32295 non-null  object 
 7   store_location    31651 non-null  object 
 8   payment_method    31635 non-null  object 
 9   discount_applied  31684 non-null  float64
dtypes: float64(3), object(7)
memory usage: 2.5+ MB


### Reviews

In [27]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   review_id         1000 non-null   object
 1   customer_id       1000 non-null   object
 2   product_name      976 non-null    object
 3   product_category  981 non-null    object
 4   full_name         981 non-null    object
 5   transaction_date  1000 non-null   object
 6   review_date       1000 non-null   object
 7   rating            1000 non-null   int64 
 8   review_title      1000 non-null   object
 9   review_text       1000 non-null   object
dtypes: int64(1), object(9)
memory usage: 78.3+ KB


### Interactions

In [28]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   interaction_id    100000 non-null  object 
 1   customer_id       100000 non-null  object 
 2   channel           97998 non-null   object 
 3   interaction_type  97978 non-null   object 
 4   interaction_date  100000 non-null  object 
 5   duration          98037 non-null   float64
 6   page_or_product   98073 non-null   object 
 7   session_id        100000 non-null  object 
dtypes: float64(1), object(7)
memory usage: 6.1+ MB


### Customers

In [29]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        5000 non-null   object 
 1   full_name          4894 non-null   object 
 2   age                4814 non-null   float64
 3   gender             4888 non-null   object 
 4   email              4889 non-null   object 
 5   phone              4814 non-null   object 
 6   street_address     4823 non-null   object 
 7   city               4903 non-null   object 
 8   state              4907 non-null   object 
 9   zip_code           4904 non-null   float64
 10  registration_date  5000 non-null   object 
 11  preferred_channel  4886 non-null   object 
dtypes: float64(2), object(10)
memory usage: 468.9+ KB
